In [1]:
#defining file path for test dataset
dataset = "IN-Abs" # Options: IN - IN-Abs, UK-UK-Abs, N2-IN-Ext
output_path = "/kaggle/working/output/"



In [2]:
cd /kaggle/input/utilis


/kaggle/input/utilis


In [3]:
#importing necessary libraries
import pandas as pd
import numpy as np
import glob
import sys
sys.path.insert(0, '../')
from utilities import *
import os
import nltk

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
if not os.path.exists(output_path):
    os.makedirs(output_path)
names, data_source, data_summary = get_summary_data(dataset, "test")

In [5]:
#references is a list containg all the summaries in the test dataset
references=list()
for reference_summary in data_summary:
    references.append(reference_summary)

In [6]:
!pip install evaluate
!pip install rouge_score

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=85cc4383ca10e07503bc7afd63c016046b6b1a8869d6cad6b9d82c72361cb002
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [7]:
import torch
#comapring wirh original legal pegasus
base_pegasus_model_candidates=list()
#loading gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
base_pegasus_model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus")
base_pegasus_tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus") 
base_pegasus_model.to(device)


for text in data_source[:50]:
    base_pegasus_input_tokenized = base_pegasus_tokenizer.encode(text, return_tensors='pt', max_length=1024, truncation=True)
    base_pegasus_input_tokenized = base_pegasus_input_tokenized.to(device)  # Move input tensor to GPU if available

    # Ensure the model and input tensors are on the same device
    with torch.no_grad():
        base_pegasus_summary_ids =base_pegasus_model.generate(base_pegasus_input_tokenized,
                                                num_beams=9,
                                                no_repeat_ngram_size=3,
                                                length_penalty=2.0,
                                                min_length=200,
                                                max_length=1000,
                                                early_stopping=True)

    base_pegasus_summary = [base_pegasus_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False)
               for g in base_pegasus_summary_ids][0]
    base_pegasus_model_candidates.append(base_pegasus_summary)

In [8]:
from evaluate import load
# Load the ROUGE metric
import evaluate

rouge = evaluate.load('rouge')
results = rouge.compute(predictions=base_pegasus_model_candidates, references=references[:50])
print(results)

{'rouge1': 0.3718602595930274, 'rouge2': 0.16267836219359194, 'rougeL': 0.20150110761311663, 'rougeLsum': 0.32028189520868344}


In [9]:
from transformers import AutoTokenizer, BartForConditionalGeneration
base_bart_model_candidates=list()

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")

for text in data_source[:50]:
    inputs = tokenizer([text], max_length=1024, return_tensors="pt")

# Generate Summary
    summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=200, max_length=1000)

    tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  
    base_bart_model_candidates.append(summary)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [10]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=base_bart_model_candidates, references=references[:50])
print(results)

{'rouge1': 0.3928007666136927, 'rouge2': 0.1603038307991722, 'rougeL': 0.19450293647610467, 'rougeLsum': 0.33602668662181145}
